# Домашнее задание по Автобрее №2
## Настя Чижикова, БКЛ181

In [1]:
import gensim
import pandas as pd
from pprint import pprint
import re 
import spacy
from tqdm.auto import tqdm
import nltk
import numpy as np
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Анастасия\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

In [46]:
df = pd.read_json('https://raw.githubusercontent.com/selva86/datasets/master/newsgroups.json')
print(df.target_names.unique())
df.head()

['rec.autos' 'comp.sys.mac.hardware' 'rec.motorcycles' 'misc.forsale'
 'comp.os.ms-windows.misc' 'alt.atheism' 'comp.graphics'
 'rec.sport.baseball' 'rec.sport.hockey' 'sci.electronics' 'sci.space'
 'talk.politics.misc' 'sci.med' 'talk.politics.mideast'
 'soc.religion.christian' 'comp.windows.x' 'comp.sys.ibm.pc.hardware'
 'talk.politics.guns' 'talk.religion.misc' 'sci.crypt']


,content,target,target_names
0,From: lerxst@wam.umd.edu (where's my thing)\nS...,7,rec.autos
1,From: guykuo@carson.u.washington.edu (Guy Kuo)...,4,comp.sys.mac.hardware
10,From: irwin@cmptrc.lonestar.org (Irwin Arnstei...,8,rec.motorcycles
100,From: tchen@magnus.acs.ohio-state.edu (Tsung-K...,6,misc.forsale
1000,From: dabl2@nlm.nih.gov (Don A.B. Lindbergh)\n...,2,comp.os.ms-windows.misc


In [47]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11314 entries, 0 to 9999
Data columns (total 3 columns):
content         11314 non-null object
target          11314 non-null int64
target_names    11314 non-null object
dtypes: int64(1), object(2)
memory usage: 353.6+ KB


In [48]:
data = df.content.values.tolist()

data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

data = [re.sub('\s+', ' ', sent) for sent in data]

data = [re.sub("\'", "", sent) for sent in data]

In [49]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

In [50]:
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram) 

In [51]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [52]:
data_words_nostops = remove_stopwords(data_words)

data_words_bigrams = make_bigrams(data_words_nostops)

nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

In [53]:
id2word = corpora.Dictionary(data_lemmatized)

texts = data_lemmatized

corpus = [id2word.doc2bow(text) for text in texts]

In [54]:
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('addition', 1),
  ('body', 1),
  ('bring', 1),
  ('call', 1),
  ('car', 5),
  ('could', 1),
  ('day', 1),
  ('door', 2),
  ('early', 1),
  ('engine', 1),
  ('enlighten', 1),
  ('funky', 1),
  ('history', 1),
  ('host', 1),
  ('info', 1),
  ('know', 1),
  ('late', 1),
  ('lerxst', 1),
  ('line', 1),
  ('look', 2),
  ('mail', 1),
  ('make', 1),
  ('model', 1),
  ('name', 1),
  ('neighborhood', 1),
  ('nntp_poste', 1),
  ('park', 1),
  ('production', 1),
  ('really', 1),
  ('rest', 1),
  ('see', 1),
  ('separate', 1),
  ('small', 1),
  ('spec', 1),
  ('sport', 1),
  ('thank', 1),
  ('thing', 1),
  ('where', 1),
  ('wonder', 1),
  ('year', 1)]]

#### Подберем оптимальное количество топиков 

In [69]:
def count_coherence(lda_model):
    coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
    coherence_lda = coherence_model_lda.get_coherence()
    return(coherence_lda)

In [70]:
top_num = np.arange(15, 25)

for t in tqdm(top_num):
    lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=t, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)
    
    print('Количество топиков: ', t, ', Coherence Score: ', count_coherence(lda_model))

Количество топиков:  15 , Coherence Score:  0.486535444645369
Количество топиков:  16 , Coherence Score:  0.47978458818380076
Количество топиков:  17 , Coherence Score:  0.4918424526814874
Количество топиков:  18 , Coherence Score:  0.47646871114361544
Количество топиков:  19 , Coherence Score:  0.4667167291893448
Количество топиков:  20 , Coherence Score:  0.4494927348421709
Количество топиков:  21 , Coherence Score:  0.45162683124773734
Количество топиков:  22 , Coherence Score:  0.4249598649749663
Количество топиков:  23 , Coherence Score:  0.4635667273567563
Количество топиков:  24 , Coherence Score:  0.40607726078758893



Оптимальное количество - 17

In [55]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=17, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [56]:
pprint(lda_model.show_topics(formatted=False))
doc_lda = lda_model[corpus]

[(0,
  [('trial', 0.07918365),
   ('safety', 0.07534665),
   ('pin', 0.06717999),
   ('panel', 0.060179982),
   ('identity', 0.02560642),
   ('warm', 0.016183434),
   ('vw', 0.01123607),
   ('ground', 1.6851787e-05),
   ('wire', 1.6845013e-05),
   ('witness', 1.6806922e-05)]),
 (8,
  [('ax', 0.9314803),
   ('display', 0.009234823),
   ('wide', 0.004168393),
   ('menu', 0.0021082633),
   ('greatly_appreciate', 0.0020101422),
   ('hi', 0.001597008),
   ('mb', 0.0010460928),
   ('xlib', 0.0010023705),
   ('cursor', 0.0007968963),
   ('liter', 0.0005393434)]),
 (15,
  [('hour', 0.1250762),
   ('multiple', 0.043241654),
   ('clock', 0.038952228),
   ('massacre', 0.031672567),
   ('pointer', 0.029385818),
   ('bag', 0.02812787),
   ('resort', 0.025504714),
   ('newsletter', 0.024846807),
   ('rush', 0.021762604),
   ('dodger', 0.021430545)]),
 (11,
  [('drug', 0.12400299),
   ('doctor', 0.0747598),
   ('lab', 0.044118498),
   ('acquire', 0.033091918),
   ('programmer', 0.02826848),
   ('suit

#### Функция, возвращающая самый широкий топик

In [57]:
lda_model.show_topics(num_topics=17, formatted=False)[0]

(0,
 [('trial', 0.07918365),
  ('safety', 0.07534665),
  ('pin', 0.06717999),
  ('panel', 0.060179982),
  ('identity', 0.02560642),
  ('warm', 0.016183434),
  ('vw', 0.01123607),
  ('ground', 1.6851787e-05),
  ('wire', 1.6845013e-05),
  ('witness', 1.6806922e-05)])

Сделаем матрицу весов для всех слов в каждом топике, чтобы потом перемножить и посчитать сумму

In [58]:
topic_df = pd.DataFrame(index=[i for i in range(17)])

for t, words in lda_model.show_topics(num_topics=17, formatted=False):
    for w, value in words:
        if w not in topic_df.columns:
            topic_df[w] = [0.0] * 17
            
        topic_df[w][t] = value
        
topic_df.head()

,trial,safety,pin,panel,identity,warm,vw,ground,wire,witness,...,dodger,line,host,nntp_poste,organization,thank,new,need,also,reply
0,0.079184,0.075347,0.06718,0.06018,0.025606,0.016183,0.011236,0.000017,0.000017,0.000017,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [59]:
topic_df = topic_df.reindex(sorted(topic_df.columns), axis=1)
topic_df.head()

,acquire,also,arrest,article,available,ax,bag,band,be,become,...,wide,win,window,wire,witness,work,would,write,xlib,year
0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.000017,0.000017,0.000000,0.0,0.0,0.0,0.0
1,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.019316,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
2,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.022045,0.000000,0.000000,0.021256,0.0,0.0,0.0,0.0
3,0.0,0.0,0.019258,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
4,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0


In [60]:
cnt_texts = []

for c in texts:
    text = ' '.join(c)
    cnt_texts.append(text)

Получаем матрицу, в которой подсчитаны вхождения каждого слова в каждый текст (но оставим только те слова, которые есть в списке топиков)

In [61]:
from sklearn.feature_extraction.text import CountVectorizer
cnt_vec = CountVectorizer(vocabulary=topic_df.columns)
X = cnt_vec.fit_transform(cnt_texts)
X.toarray()

array([[0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 2, 0, ..., 0, 0, 0]], dtype=int64)

In [62]:
X.shape

(11314, 165)

In [63]:
df = pd.DataFrame(X.todense(), columns=cnt_vec.get_feature_names())
df.head()

,acquire,also,arrest,article,available,ax,bag,band,be,become,...,wide,win,window,wire,witness,work,would,write,xlib,year
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,0,0,0,0,1,0,0,0,0,0,...,0,0,6,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [64]:
df['top_topic'] = np.argmax(df.to_numpy().dot(topic_df.to_numpy().T), axis=1)
df.head()

,acquire,also,arrest,article,available,ax,bag,band,be,become,...,win,window,wire,witness,work,would,write,xlib,year,top_topic
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,16
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,15
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,16
3,0,0,0,0,1,0,0,0,0,0,...,0,6,0,0,0,0,0,0,0,2
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,16


In [65]:
df['text'] = cnt_texts
df.head()

,acquire,also,arrest,article,available,ax,bag,band,be,become,...,window,wire,witness,work,would,write,xlib,year,top_topic,text
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,16,where thing car nntp_poste host park line wond...
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,15,si poll final summary final call si clock repo...
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,16,s worth distribution_usa may gmt organization ...
3,0,0,0,0,1,0,0,0,0,0,...,6,0,0,0,0,0,0,0,2,lot nntp_poste organization post friend call d...
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,16,organization national medicine line see mouse_...


In [66]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [67]:
for _, group in df.groupby('top_topic'):
    text_ids = group.index.values
    texts = [cnt_texts[i] for i in text_ids]
    vectorizer = TfidfVectorizer(max_df=0.98, analyzer=str.split, max_features=5000)
    X = vectorizer.fit_transform(texts).toarray()
    word_ids = X.argsort().T[-5:].T
    features = vectorizer.get_feature_names()
    
    def id_to_word(id_list):
        words = [features[i] for i in id_list]
        return ", ".join(words)
    
    all_words = np.apply_along_axis(id_to_word, axis=1, arr=word_ids)
    
    for text_id, words in zip(text_ids, all_words):
        df.loc[text_id, 'top_words'] = words

In [68]:
final_df = df[['text', 'top_topic', 'top_words']]
final_df.head()

,text,top_topic,top_words
0,where thing car nntp_poste host park line wond...,16,"neighborhood, where, enlighten, door, car"
1,si poll final summary final call si clock repo...,15,"add, final, si, poll, clock"
2,s worth distribution_usa may gmt organization ...,16,"gmt, stable, leak, fade, bike"
3,lot nntp_poste organization post friend call d...,2,"commonview, code, software, framework, window"
4,organization national medicine line see mouse_...,16,"indication, diamond, distortion, mouse_cu"


In [30]:
mallet_path = 'C:/anaconda/mallet-2.0.8/bin/mallet' 
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=20, id2word=id2word)

CalledProcessError: Command 'C:/anaconda/mallet-2.0.8/bin/mallet import-file --preserve-case --keep-sequence --remove-stopwords --token-regex "\S+" --input C:\temp\e8f8d5_corpus.txt --output C:\temp\e8f8d5_corpus.mallet' returned non-zero exit status 1.

#### У меня та же самая ошибка, что и у остальных, не снижайте пожалста (в колабе тоже не работает)